In [2]:
import datetime
import json
import numpy as np
from geopy.distance import geodesic
import requests
import csv
import os
import logging
from collections import defaultdict

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class Config:
    FRAUD_TRANSACTIONS_FILE = "fraud_transactions.csv"
    VALID_TRANSACTIONS_FILE = "valid_transactions.csv"
    FRAUD_CASES_FILE = "fraud_cases.json"
    BLACKLISTED_COUNTRIES = {"North Korea", "Iran", "Syria", "Cuba"}
    IP_GEOLOCATION_API_URL = "http://ip-api.com/json/"
    IMPOSSIBLE_TRAVEL_SPEED_THRESHOLD = 800  # km/h
    AMOUNT_Z_SCORE_THRESHOLD = 3
    TRANSACTION_VELOCITY_THRESHOLD = 5  # transactions per hour
    MIN_COUNTRY_CHANGE_HOURS = 1
    GEOLOCATION_CACHE_TTL = 3600  # 1 hour in seconds
    USUAL_LOCATION_MIN_TRANSACTIONS = 3
    USUAL_LOCATION_DAYS_WINDOW = 30
    ALLOWED_CITY_VARIANCE = 2

class IPGeolocationCache:
    def __init__(self):
        self.cache = {}

    def get(self, ip_address):
        cached_entry = self.cache.get(ip_address)
        if cached_entry and (datetime.datetime.now() - cached_entry['timestamp']).seconds < Config.GEOLOCATION_CACHE_TTL:
            return cached_entry['data']
        
        try:
            response = requests.get(f"{Config.IP_GEOLOCATION_API_URL}{ip_address}", timeout=3)
            if response.status_code == 200:
                data = response.json()
                if data.get('status') == 'success':
                    geolocation_data = {
                        'city': data.get('city', 'Unknown'),
                        'country': data.get('country', 'Unknown'),
                        'latitude': data.get('lat', 0),
                        'longitude': data.get('lon', 0),
                        'timezone': data.get('timezone', 'Unknown')
                    }
                    self.cache[ip_address] = {
                        'data': geolocation_data,
                        'timestamp': datetime.datetime.now()
                    }
                    return geolocation_data
                else:
                    logging.error(f"IP API error: {data.get('message')}")
            else:
                logging.error(f"IP API request failed with status {response.status_code}")
        except Exception as e:
            logging.error(f"Error fetching geolocation: {e}")
        return None

class Transaction:
    ip_cache = IPGeolocationCache()

    def __init__(self, transaction_id, user_id, amount, timestamp, ip_address, device_id):
        self.transaction_id = transaction_id
        self.user_id = user_id
        self.amount = float(amount)
        
        try:
            self.timestamp = datetime.datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
        except ValueError as e:
            logging.error(f"Invalid timestamp format: {timestamp}")
            raise
        
        self.ip_address = ip_address
        self.device_id = device_id
        
        ip_data = self.ip_cache.get(ip_address) or {}
        self.country = ip_data.get('country', 'Unknown')
        self.city = ip_data.get('city', 'Unknown')
        self.latitude = ip_data.get('latitude', 0)
        self.longitude = ip_data.get('longitude', 0)
        self.timezone = ip_data.get('timezone', 'Unknown')

class FraudDetector:
    def __init__(self):
        self.fraud_cases = []
        self.user_transaction_history = defaultdict(lambda: {
            'timestamps': [],
            'amounts': [],
            'cities': [],
            'countries': [],
            'devices': [],
            'ips': [],
            'timezones': [],
            'locations': []
        })
        self.blacklisted_devices = set()
        self.blacklisted_ips = set()
        self.load_transaction_history()

    def get_usual_location(self, user_id):
        """Determine user's usual location based on recent history"""
        history = self.user_transaction_history[user_id]
        window_start = datetime.datetime.now() - datetime.timedelta(days=Config.USUAL_LOCATION_DAYS_WINDOW)
        
        recent_locations = [
            (city, country, ts) 
            for city, country, ts in zip(history['cities'], history['countries'], history['timestamps'])
            if ts > window_start
        ]
        
        if len(recent_locations) < Config.USUAL_LOCATION_MIN_TRANSACTIONS:
            return None
        
        location_counts = defaultdict(int)
        for city, country, _ in recent_locations:
            location_counts[(city, country)] += 1
        
        (usual_city, usual_country), _ = max(location_counts.items(), key=lambda x: x[1], default=(None, None))
        return usual_city, usual_country

    def load_transaction_history(self):
        for file_name in [Config.FRAUD_TRANSACTIONS_FILE, Config.VALID_TRANSACTIONS_FILE]:
            if not os.path.exists(file_name):
                continue

            try:
                with open(file_name, mode='r') as file:
                    reader = csv.DictReader(file)
                    for row in reader:
                        user_id = int(row["user_id"])
                        self.user_transaction_history[user_id]['timestamps'].append(
                            datetime.datetime.strptime(row["timestamp"], "%Y-%m-%d %H:%M:%S")
                        )
                        self.user_transaction_history[user_id]['amounts'].append(float(row["amount"]))
                        self.user_transaction_history[user_id]['locations'].append((float(row["latitude"]), float(row["longitude"])))
                        self.user_transaction_history[user_id]['devices'].append(row["device_id"])
                        self.user_transaction_history[user_id]['ips'].append(row["ip_address"])
                        self.user_transaction_history[user_id]['countries'].append(row["country"])
                        self.user_transaction_history[user_id]['cities'].append(row.get("city", "Unknown"))
            except Exception as e:
                logging.error(f"Error loading transaction history from {file_name}: {e}")

    def save_transaction(self, transaction, is_fraud, reasons):
        try:
            file_name = Config.FRAUD_TRANSACTIONS_FILE if is_fraud else Config.VALID_TRANSACTIONS_FILE
            file_exists = os.path.exists(file_name)
            
            with open(file_name, mode='a', newline='') as file:
                writer = csv.writer(file)
                if not file_exists:
                    writer.writerow([
                        "user_id", "timestamp", "amount", "latitude", 
                        "longitude", "device_id", "ip_address", 
                        "country", "city", "reasons"
                    ])
                writer.writerow([
                    transaction.user_id,
                    transaction.timestamp.strftime("%Y-%m-%d %H:%M:%S"),
                    transaction.amount,
                    transaction.latitude,
                    transaction.longitude,
                    transaction.device_id,
                    transaction.ip_address,
                    transaction.country,
                    transaction.city,
                    "; ".join(reasons)
                ])
        except IOError as e:
            logging.error(f"Failed to save transaction: {e}")

    def detect_fraud(self, transaction):
        reasons = []
        user_history = self.user_transaction_history[transaction.user_id]
        usual_location = self.get_usual_location(transaction.user_id)

        if transaction.country in Config.BLACKLISTED_COUNTRIES:
            reasons.append(f"Restricted country: {transaction.country}")
            
        if transaction.device_id in self.blacklisted_devices:
            reasons.append(f"Blacklisted device: {transaction.device_id}")
        if transaction.ip_address in self.blacklisted_ips:
            reasons.append(f"Blacklisted IP: {transaction.ip_address}")

        if usual_location:
            usual_city, usual_country = usual_location
            if transaction.country != usual_country:
                reasons.append(f"Country deviation from usual {usual_country}")
            if transaction.city != usual_city:
                reasons.append(f"City deviation from usual {usual_city}")

        if user_history['timestamps']:
            if transaction.device_id not in user_history['devices']:
                reasons.append(f"New device detected: {transaction.device_id}")
          
            if transaction.ip_address != user_history['ips'][-1]:
                reasons.append("IP address changed")

            amounts = user_history['amounts']
            if len(amounts) > 1:
                mean_amount = np.mean(amounts)
                std_amount = np.std(amounts)
                if std_amount > 0:
                    z_score = (transaction.amount - mean_amount) / std_amount
                    if abs(z_score) > Config.AMOUNT_Z_SCORE_THRESHOLD:
                        reasons.append(f"Amount anomaly: {transaction.amount:.2f} (mean: {mean_amount:.2f}, σ: {std_amount:.2f})")
                else:
                    logging.warning(f"Standard deviation is zero for user {transaction.user_id}")

            recent_count = sum(1 for ts in user_history['timestamps'] 
                            if (transaction.timestamp - ts).total_seconds() <= 3600)
            if recent_count >= Config.TRANSACTION_VELOCITY_THRESHOLD:
                reasons.append(f"High transaction velocity ({recent_count} in last hour)")

            current_location = (transaction.latitude, transaction.longitude)
            for past_ts, past_loc in zip(user_history['timestamps'], user_history['locations']):
                if past_loc == (0, 0):
                    continue
                time_diff = (transaction.timestamp - past_ts).total_seconds() / 3600
                if time_diff > 0:
                    distance = geodesic(past_loc, current_location).km
                    speed = distance / time_diff
                    if speed > Config.IMPOSSIBLE_TRAVEL_SPEED_THRESHOLD:
                        reasons.append(f"Impossible travel: {distance:.1f}km in {time_diff:.1f}h")
                        break

            if user_history['countries']:
                last_country = user_history['countries'][-1]
                time_diff = (transaction.timestamp - user_history['timestamps'][-1]).total_seconds() / 3600
                if transaction.country != last_country and time_diff < Config.MIN_COUNTRY_CHANGE_HOURS:
                    reasons.append(f"Rapid country change from {last_country} to {transaction.country}")

        user_history['timestamps'].append(transaction.timestamp)
        user_history['amounts'].append(transaction.amount)
        user_history['locations'].append((transaction.latitude, transaction.longitude))
        user_history['devices'].append(transaction.device_id)
        user_history['ips'].append(transaction.ip_address)
        user_history['countries'].append(transaction.country)
        user_history['cities'].append(transaction.city)

        is_fraud = bool(reasons)
        if not is_fraud:
            reasons = ["Normal transaction"]
            
        self.save_transaction(transaction, is_fraud, reasons)

        if is_fraud:
            self.fraud_cases.append({
                'transaction_id': transaction.transaction_id,
                'user_id': transaction.user_id,
                'amount': transaction.amount,
                'reasons': reasons,
                'timestamp': transaction.timestamp.isoformat(),
                'country': transaction.country,
                'device': transaction.device_id,
                'ip': transaction.ip_address
            })
            self.blacklisted_devices.add(transaction.device_id)
            self.blacklisted_ips.add(transaction.ip_address)
            return True, reasons
        
        return False, reasons

    def save_fraud_cases(self):
        try:
            with open(Config.FRAUD_CASES_FILE, "w") as file:
                json.dump(self.fraud_cases, file, indent=4, default=str)
            logging.info(f"Saved {len(self.fraud_cases)} fraud cases to {Config.FRAUD_CASES_FILE}")
        except IOError as e:
            logging.error(f"Failed to save fraud cases: {e}")

if __name__ == "__main__":
    test_transactions = [
        Transaction(101, 123, 15000, "2025-03-30 12:00:00", "8.8.8.8", "device123"),
        Transaction(102, 123, 12000, "2025-03-30 12:05:00", "8.8.8.8", "device124"),
        Transaction(103, 456, 50000, "2025-03-30 12:10:00", "1.1.1.1", "hacked_device"),
        Transaction(104, 123, 200, "2025-03-30 14:00:00", "9.9.9.9", "device125"),
        Transaction(105, 234, 500, "2025-03-30 15:00:00", "10.10.10.10", "device126"),
        Transaction(106, 345, 1000, "2025-03-30 16:00:00", "11.11.11.11", "device127"),
        Transaction(107, 456, 1500, "2025-03-30 16:10:00", "12.12.12.12", "device128"),
        Transaction(108, 456, 1600, "2025-03-30 16:15:00", "13.13.13.13", "device128"),
        Transaction(109, 567, 2500, "2025-03-30 17:00:00", "14.14.14.14", "new_device001"),
        Transaction(110, 678, 90000, "2025-03-30 18:00:00", "15.15.15.15", "device130"),
        Transaction(111, 789, 100, "2025-03-30 19:00:00", "16.16.16.16", "device131"),
        Transaction(112, 789, 110, "2025-03-30 19:05:00", "16.16.16.16", "device131"),
        Transaction(113, 789, 120, "2025-03-30 19:10:00", "16.16.16.16", "device131"),
        Transaction(114, 789, 130, "2025-03-30 19:15:00", "16.16.16.16", "device131"),
        Transaction(115, 789, 140, "2025-03-30 19:20:00", "16.16.16.16", "device131"),
        Transaction(116, 890, 3000, "2025-03-30 20:00:00", "17.17.17.17", "hacked_device"),
        Transaction(117, 901, 4500, "2025-03-30 21:00:00", "18.18.18.18", "device132"),
        Transaction(118, 902, 6000, "2025-03-30 03:00:00", "19.19.19.19", "device133"),
        Transaction(119, 903, 700, "2025-03-30 22:00:00", "20.20.20.20", "device134"),
        Transaction(120, 903, 800, "2025-03-30 22:10:00", "21.21.21.21", "device134")
    ]
    
    fraud_detector = FraudDetector()
    
    for transaction in test_transactions:
        try:
            is_fraud, reasons = fraud_detector.detect_fraud(transaction)
            status = "Fraud" if is_fraud else "Valid"
            logging.info(f"Transaction {transaction.transaction_id} - {status}")
            logging.info(f"Reasons: {reasons}\n")
        except Exception as e:
            logging.error(f"Error processing transaction {transaction.transaction_id}: {e}")
    
    fraud_detector.save_fraud_cases()

2025-03-31 11:58:37,936 - ERROR - IP API error: private range
2025-03-31 11:58:45,514 - INFO - Transaction 101 - Valid
2025-03-31 11:58:45,516 - INFO - Reasons: ['Normal transaction']

2025-03-31 11:58:45,521 - INFO - Transaction 102 - Fraud
2025-03-31 11:58:45,525 - INFO - Reasons: ['New device detected: device124']

2025-03-31 11:58:45,530 - INFO - Transaction 103 - Valid
2025-03-31 11:58:45,532 - INFO - Reasons: ['Normal transaction']

2025-03-31 11:58:45,543 - INFO - Transaction 104 - Fraud
2025-03-31 11:58:45,547 - INFO - Reasons: ['New device detected: device125', 'IP address changed', 'Amount anomaly: 200.00 (mean: 13500.00, σ: 1500.00)', 'Impossible travel: 3869.8km in 2.0h']

2025-03-31 11:58:45,553 - INFO - Transaction 105 - Valid
2025-03-31 11:58:45,559 - INFO - Reasons: ['Normal transaction']

2025-03-31 11:58:45,566 - INFO - Transaction 106 - Valid
2025-03-31 11:58:45,570 - INFO - Reasons: ['Normal transaction']

2025-03-31 11:58:45,582 - INFO - Transaction 107 - Fraud
202